In [3]:
using UncNLPTestSet, LinearAlgebra
using ForwardDiff

┌ Info: Precompiling UncNLPTestSet [ac41c6d5-581c-4fd8-9896-caf0766f302e]
└ @ Base loading.jl:1342
┌ Warning: TODO: Debug BDQRTIC
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BDQRTIC.jl:73
┌ Warning: TODO: Issue in CUTEst? See comment https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/broydn7d.jl#L50 
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BROYDN7D.jl:109
┌ Warning: TODO: Issue in CUTEst? See https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/brybnd.jl
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BRYBND.jl:110
┌ Warning: TODO: Debug CRAGGLVY
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/CRAGGLVY.jl:75
┌ Warning: TODO: Clean up DIXMAANB (and others) implementation. See comment block
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXMAANB.jl:108
┌ Warning: TODO: Implement DIXON3DQ
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXON3DQ.jl:1
┌ Warning: TOD

In [1]:
function gAD!(nlp, x::Vector{<:Real}, S::Matrix{<:Real}, g::Union{Vector{<:Real}, Nothing}=nothing)
	S_dual = ForwardDiff.Dual{:tag}.(x,  eachcol(S)...)
	# Y_dual = isa(g, Nothing) ? similar(S_dual) : ForwardDiff.Dual{1}.(zeros(nlp.n),  eachcol(S)...) 
	Y_dual = ForwardDiff.Dual{:tag}.(zeros(nlp.n),  eachcol(S)...) 
	# ... when removing g+= statements, we can have Y_dual = similar(S_dual)

	nlp.g!(Y_dual, S_dual)

	Yi = similar(S)
	# update forwardDiff to make extraction more economical
    @views for i in 1:nlp.n
        Yi[i, :] .= Y_dual[i].partials[:]
    end

	if !isa(g, Nothing)
		# Also this should be more economical
		for i in 1:nlp.n
			g[i] = Y_dual[i].value
		end
	end
	return Yi
end

# bDim corresponds to the number of processors available
# TODO: we are making the assumption that mod(bDim, nlp.n) ≠ 0
# TODO: use @view macro when passing S and Y to gAD? should be inplace... 
function gHS(nlp, x, S, bDim::Int)
	nlp.n < bDim && @warn("Block size $bDim ≥ $(nlp.n)/2, the problems dimension")
	bDim = Int(min(nlp.n/2, bDim)) # ensures 2 iterations of gAD to get J(∇f(x))

	# determine the first set of m-1 directions
	m = Int(mod(nlp.n, bDim))
	
	# we determine g on the first iteration
	g = similar(x)
	Y = gAD!(nlp, x, S[:, 1:(m-1)], g)

	# overwrite the last column of S to contain g
	S[:, nlp.n] = g

	for i in m:bDim:(nlp.n-bDim) 
		Yi = gAD!(nlp, x, S[:, i:(i+bDim)])
		Y = [Y Yi] 
	end
	return g, Y
end

gHS (generic function with 1 method)

In [4]:
nlp = SelectProgram("WOODS")
adjdim!(nlp, 500);

┌ Warning: This operation may be unstable
└ @ UncNLPTestSet /Users/daniel/.julia/dev/UncNLPTestSet/src/UncNLPTestSet.jl:182


In [5]:
M = rand(nlp.n, nlp.n)
S = Matrix(qr(M).Q);
Y = similar(S);

In [6]:
g, res = gHS(nlp, nlp.x0, S, 16)

([-12008.0, -2080.0, -10808.0, -1880.0, -12008.0, -2080.0, -10808.0, -1880.0, -12008.0, -2080.0  …  -10808.0, -1880.0, -12008.0, -2080.0, -10808.0, -1880.0, -12008.0, -2080.0, -10808.0, -1880.0], [-691.2478007976455 277.1729423462207 … 436.945780112063 -1.37009616e8; -78.41487093206784 28.12048492293934 … 53.8792338987941 -1.490484e7; … ; -399.74543996231483 -400.80878092360655 … -152.82009777542604 -1.10996656e8; -43.28247874464756 -48.78492884810491 … -13.555973623083654 -1.20902e7])

In [7]:
g

500-element Vector{Float64}:
 -12008.0
  -2080.0
 -10808.0
  -1880.0
 -12008.0
  -2080.0
 -10808.0
  -1880.0
 -12008.0
  -2080.0
 -10808.0
  -1880.0
 -12008.0
      ⋮
 -12008.0
  -2080.0
 -10808.0
  -1880.0
 -12008.0
  -2080.0
 -10808.0
  -1880.0
 -12008.0
  -2080.0
 -10808.0
  -1880.0

In [8]:
res

500×530 Matrix{Float64}:
 -691.248     277.173    -264.973   -751.15    …    436.946     -1.3701e8
  -78.4149     28.1205    -25.3233   -81.19          53.8792    -1.49048e7
 -766.403     807.217    -439.977    276.909       -631.287     -1.10997e8
  -88.7419     89.7539    -47.2985    28.3349       -63.202     -1.20902e7
  -92.4798   -930.797     143.314   -450.218        637.097     -1.3701e8
  -16.139    -100.988      11.768    -52.8616  …     74.3248    -1.49048e7
 -307.533     110.384     234.665   -138.084      -1013.63      -1.10997e8
  -37.236       7.32402    21.4638   -21.933       -103.657     -1.20902e7
 -309.712     160.762    -823.353    140.676        173.845     -1.3701e8
  -37.9333     12.1811    -85.7393    17.1374        13.9957    -1.49048e7
 -103.893    -693.061    -723.888    115.102   …    584.523     -1.10997e8
  -12.2486    -84.6596    -75.9289    11.1957        63.4355    -1.20902e7
 -265.913    -171.244    -471.869   -162.769       -126.752     -1.3701e8
    

In [9]:
H = hessAD(nlp, nlp.x0)

500×500 Matrix{Float64}:
 11202.0  1200.0      0.0     0.0  …      0.0     0.0      0.0     0.0
  1200.0   220.2      0.0    19.8         0.0     0.0      0.0     0.0
     0.0     0.0  10082.0  1080.0         0.0     0.0      0.0     0.0
     0.0    19.8   1080.0   200.2         0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0         0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0  …      0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0         0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0         0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0         0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0         0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0  …      0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0         0.0     0.0      0.0     0.0
     0.0     0.0      0.0     0.0         0.0     0.0      0.0     0.0
     ⋮                             ⋱                

In [10]:
H*S[:, nlp.n]

500-element Vector{Float64}:
 -1.37009616e8
 -1.490484e7
 -1.10996656e8
 -1.20902e7
 -1.37009616e8
 -1.490484e7
 -1.10996656e8
 -1.20902e7
 -1.37009616e8
 -1.490484e7
 -1.10996656e8
 -1.20902e7
 -1.37009616e8
  ⋮
 -1.37009616e8
 -1.490484e7
 -1.10996656e8
 -1.20902e7
 -1.37009616e8
 -1.490484e7
 -1.10996656e8
 -1.20902e7
 -1.37009616e8
 -1.490484e7
 -1.10996656e8
 -1.20902e7